# Print dataset in format for model

In [2]:
import pandas as pd
import numpy as np2
import math
import matplotlib.pyplot as plt
import datetime as dt
import time
import random
from refainedLibrary_v3 import *
from statsmodels.tsa.api import Holt
from scipy.interpolate import *

from IPython.display import clear_output

from tqdm.notebook import tqdm



trajectories_2_resampled_04s SINGLE SMOOTH

trajectories_2_resampled_04s_bis DOPPIO SMOOTH

In [3]:
rd = Refained_Data('sCREEN-dataset.txt')
#trajectories = rd.read_trajectories("trajectories_2_resampled_04s_bis.csv")
trajectories = rd.read_trajectories("trajectories_2_resampled_1s_shifted.csv")

In [4]:
len(trajectories)

4283

In [8]:
# DEPRECATED

"""
    Set the points in format for the models
    data: dataframe of points order by "time"
    time_delta: number of seconds for frame. Every frame is bigger as time_delta and inside it there are a certain number of point
    
    RETURN: dataframe with n collumn (n = # of frame) and 4 rows (n_frame, n_traj, x, y)
"""

def get_data_in_format(data, time_delta):
    return_df = pd.DataFrame(columns = ["n_frame", "n_traj", "x", "y"])
    end = st = 0
    n_frame = 0
    
    for i in tqdm(range(len(data))):
        delta_time = data.iloc[i]["time"] - data.iloc[st]["time"]
        
        if delta_time.seconds <= time_delta - 1:
            end = i
        else:
            time_frame = data.iloc[st:end + 1]                                  
            time_frame.drop(["id", "tag_id", "time"], axis = 1, inplace = True)
            time_frame["n_frame"] = n_frame
            return_df = return_df.append(time_frame)                    
            n_frame += 1            
            end = st = i
        
        if i == len(data)-1:
            time_frame = data.iloc[st:end + 1]                                  
            time_frame.drop(["id", "tag_id", "time"], axis = 1, inplace = True)
            time_frame["n_frame"] = n_frame
            return_df = return_df.append(time_frame)      

    return return_df.T


"""
    Given a list return the data ready for the model
    trajecotries: list of dataframe. every element inside the list is a trajectory
    thrs: treshold. How many traj for training and how many for test set
    
    RETURN: two dataframe. The first is the training set, the second is the test set
"""

def get_data_4_model(trajectories, thrs):
    #trajectories = rd.read_trajectories(PATH)
        
    resampled_traj = list(map(lambda x: x.iloc[::2], trajectories))
    all_trajectories = pd.concat(resampled_traj)
    
    max_n = 1000
    n_4_train = random.sample(range(0,max_n+1), int(max_n*thrs))
    n_4_test = [x for x in range(max_n) if x not in n_4_train]   

    train = all_trajectories[all_trajectories["n_traj"].isin(n_4_train)]
    train = train.sort_values('time').reset_index(drop=True)
    test = all_trajectories[all_trajectories["n_traj"].isin(n_4_test)]
    test = test.sort_values('time').reset_index(drop=True)
    
    train_data = get_data_in_format(train, 2)
    test_data = get_data_in_format(test, 2)
    
    print(n_4_train)
    print(n_4_test)
    
    return train_data, test_data


In [ ]:
n_4_train = [27, 375, 855, 418, 494, 155, 387, 72, 761, 322, 632, 105, 707, 216, 189, 837, 714, 637, 720, 102, 763, 486, 265, 575, 944, 470, 610, 33, 223, 71, 676, 294, 424, 843, 733, 315, 542, 678, 998, 159, 850, 164, 729, 454, 281, 19, 764, 895, 749, 514, 667, 563, 926, 701, 948, 280, 510, 721, 383, 565, 796, 726, 661, 854, 451, 672, 461, 662, 521, 849, 477, 4, 419, 108, 480, 668, 917, 256, 560, 857, 392, 278, 459, 644, 285, 879, 207, 196, 329, 231, 943, 533, 420, 253, 396, 446, 183, 856, 436, 653, 983, 212, 705, 484, 24, 753, 427, 463, 112, 364, 803, 145, 327, 469, 195, 977, 935, 807, 97, 267, 411, 633, 262, 15, 810, 896, 464, 673, 61, 670, 127, 111, 773, 509, 430, 621, 694, 890, 185, 912, 273, 408, 846, 601, 985, 723, 180, 40, 296, 945, 402, 94, 822, 762, 940, 14, 299, 30, 900, 959, 175, 326, 276, 790, 32, 76, 320, 608, 9, 248, 274, 738, 37, 176, 540, 232, 138, 899, 305, 150, 354, 656, 934, 6, 643, 567, 630, 113, 671, 252, 894, 902, 66, 871, 526, 980, 68, 674, 536, 853, 544, 18, 204, 357, 523, 747, 590, 336, 814, 445, 695, 126, 441, 901, 571, 47, 246, 539, 972, 892, 782, 239, 548, 214, 792, 473, 114, 46, 524, 642, 995, 646, 949, 429, 335, 860, 266, 905, 226, 946, 641, 168, 908, 817, 845, 190, 765, 74, 684, 868, 928, 130, 657, 566, 602, 652, 927, 920, 75, 541, 260, 677, 639, 768, 341, 755, 323, 468, 953, 457, 611, 829, 89, 893, 347, 623, 906, 440, 727, 936, 620, 472, 227, 597, 404, 505, 604, 629, 797, 852, 337, 706, 585, 415, 499, 914, 497, 498, 704, 546, 235, 358, 433, 194, 5, 99, 307, 691, 182, 350, 699, 49, 91, 425, 780, 960, 703, 53, 913, 618, 325, 709, 401, 651, 719, 388, 579, 140, 219, 83, 587, 181, 431, 982, 416, 941, 950, 339, 289, 17, 996, 742, 172, 434, 554, 766, 166, 545, 724, 872, 751, 57, 308, 527, 136, 121, 96, 148, 213, 417, 537, 7, 839, 774, 838, 690, 251, 103, 532, 784, 507, 199, 568, 979, 580, 177, 403, 154, 836, 400, 793, 600, 487, 328, 491, 835, 616, 832, 603, 974, 758, 504, 178, 798, 100, 581, 134, 162, 80, 708, 174, 125, 69, 988, 365, 543, 376, 907, 862, 135, 635, 60, 217, 767, 467, 962, 393, 976, 422, 475, 395, 410, 737, 617, 535, 772, 197, 660, 225, 479, 625, 654, 348, 378, 50, 508, 124, 55, 811, 233, 272, 489, 881, 330, 981, 363, 615, 277, 777, 123, 502, 110, 924, 391, 994, 98, 680, 750, 316, 84, 869, 512, 379, 390, 802, 367, 844, 258, 612, 880, 81, 528, 696, 744, 882, 592, 191, 506, 583, 78, 801, 370, 978, 687, 211, 675, 343, 156, 186, 117, 353, 986, 452, 993, 147, 870, 947, 466, 717, 242, 82, 143, 483, 556, 910, 728, 606, 897, 599, 318, 414, 638, 538, 964, 369, 342, 496, 312, 965, 718, 465, 284, 649, 760, 428, 748, 770, 922, 688, 929, 282, 800, 448, 250, 698, 815, 813, 255, 971, 295, 43, 366, 681, 501, 10, 818, 865, 552, 557, 735, 90, 79, 192, 362, 669, 968, 144, 1000, 722, 31, 785, 842, 786, 288, 884, 423, 659, 958, 825, 992, 0, 702, 222, 873, 519, 787, 831, 453, 2, 969, 209, 67, 520, 756, 118, 426, 820, 804, 990, 493, 173, 791, 133, 380, 371, 848, 874, 515, 101, 823, 20, 86, 56, 549, 298, 711, 52, 655, 444, 725, 598, 942, 42, 867, 45, 591, 740, 474, 270, 221, 700, 891, 54, 613, 627, 488, 476, 344, 503, 525, 741, 937, 293, 292, 28, 626, 824, 495, 485, 679, 769, 163, 407, 70, 462, 663, 202, 500, 781, 775, 241, 1, 346, 957, 732, 809, 841, 286, 398, 883, 712, 634, 664, 149, 961, 300, 551, 847, 249, 686, 119, 151, 275, 314, 373, 692, 93, 65, 840, 697, 577, 188, 628, 319, 447, 875, 951, 799, 171, 666, 915, 833, 331, 665, 349, 819, 861, 240, 588, 87, 877, 44, 932, 23, 334, 816, 582, 851, 650, 263, 522, 745, 471, 311, 129, 116, 904, 788, 257, 573, 304, 244, 137, 555, 16, 783, 531, 229, 886, 215, 157, 456, 594, 987, 8, 412, 106, 919, 866, 805, 310, 478, 210, 443, 435, 716, 570, 193, 450, 36, 455, 888, 386, 85, 95, 421, 889, 230, 834, 77, 658, 88, 41, 887, 967, 109, 806, 885, 283, 438, 161, 991, 460, 303, 911, 713, 921, 389, 22, 247, 306, 237, 62, 636, 736, 63, 384, 152, 930, 332, 333, 399, 576, 595, 682, 624, 437, 685, 997, 966]
 n_4_test = [3, 11, 12, 13, 21, 25, 26, 29, 34, 35, 38, 39, 48, 51, 58, 59, 64, 73, 92, 104, 107, 115, 120, 122, 128, 131, 132, 139, 141, 142, 146, 153, 158, 160, 165, 167, 169, 170, 179, 184, 187, 198, 200, 201, 203, 205, 206, 208, 218, 220, 224, 228, 234, 236, 238, 243, 245, 254, 259, 261, 264, 268, 269, 271, 279, 287, 290, 291, 297, 301, 302, 309, 313, 317, 321, 324, 338, 340, 345, 351, 352, 355, 356, 359, 360, 361, 368, 372, 374, 377, 381, 382, 385, 394, 397, 405, 406, 409, 413, 432, 439, 442, 449, 458, 481, 482, 490, 492, 511, 513, 516, 517, 518, 529, 530, 534, 547, 550, 553, 558, 559, 561, 562, 564, 569, 572, 574, 578, 584, 586, 589, 593, 596, 605, 607, 609, 614, 619, 622, 631, 640, 645, 647, 648, 683, 689, 693, 710, 715, 730, 731, 734, 739, 743, 746, 752, 754, 757, 759, 771, 776, 778, 779, 789, 794, 795, 808, 812, 821, 826, 827, 828, 830, 858, 859, 863, 864, 876, 878, 898, 903, 909, 916, 918, 923, 925, 931, 933, 938, 939, 952, 954, 955, 956, 963, 970, 973, 975, 984, 989, 999]


In [5]:
"""
    Set the points in format for the models
    data: dataframe of points order by "time"
    time_delta: number of seconds for frame. Every frame is bigger as time_delta and inside it there are a certain number of point
    
    RETURN: dataframe with n collumn (n = # of frame) and 4 rows (n_frame, n_traj, x, y)
"""
def get_data_in_format(data, time_delta):
    return_df = pd.DataFrame(columns=["n_frame", "n_traj", "x", "y"])
    st = data.iloc[0]

    n_frame = 0

    dictionary_list = []

    dictionary_data = {
        "n_frame": n_frame,
        "n_traj": st["n_traj"],
        "x": st["x"],
        "y": st["y"]
    }
    dictionary_list.append(dictionary_data)


    for i in tqdm(range(1, len(data))):
        row = data.iloc[i]
        delta_time = row["time"] - st["time"]

        if delta_time.total_seconds() < time_delta:
            dictionary_data = {
                "n_frame": n_frame,
                "n_traj": row["n_traj"],
                "x": row["x"],
                "y": row["y"]
            }
            dictionary_list.append(dictionary_data)
        else:
            st = row.copy()
            n_frame += 1
            dictionary_data = {
                "n_frame": n_frame,
                "n_traj": row["n_traj"],
                "x": row["x"],
                "y": row["y"]
            }
            dictionary_list.append(dictionary_data)

    return_df = pd.DataFrame.from_dict(dictionary_list)
    return return_df.T

"""
    Given a list return the data ready for the model
    trajecotries: list of dataframe. every element inside the list is a trajectory
    thrs: treshold. How many traj for training and how many for test set
    
    RETURN: two dataframe. The first is the training set, the second is the test set
"""

def get_data_4_model(trajectories, thrs, dim_frame):

    all_trajectories = pd.concat(trajectories)
    dim = len(trajectories)
    
    max_n = 1500
    # prime 1000 traj dal giorno 10 al giorno 14 
    # 0-800 train e 801 a 1000 test
    #n_4_train = [60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 261, 262, 263, 264, 265, 266, 267, 268, 269, 270, 271, 272, 273, 274, 275, 276, 277, 278, 279, 280, 281, 282, 283, 284, 285, 286, 287, 288, 289, 290, 291, 292, 293, 294, 295, 296, 297, 298, 299, 300, 301, 302, 303, 304, 305, 306, 307, 308, 309, 310, 311, 312, 313, 314, 315, 316, 317, 318, 319, 320, 321, 322, 323, 324, 325, 326, 327, 328, 329, 330, 331, 332, 333, 334, 335, 336, 337, 338, 339, 402, 403, 404, 405, 406, 407, 408, 409, 410, 411, 412, 413, 414, 415, 416, 417, 418, 419, 420, 421, 422, 423, 424, 425, 426, 427, 428, 429, 430, 431, 432, 433, 434, 435, 436, 437, 438, 439, 440, 441, 442, 443, 444, 445, 446, 447, 448, 449, 450, 451, 452, 457, 458, 459, 460, 461, 462, 463, 464, 465, 466, 467, 468, 469, 470, 471, 472, 473, 474, 475, 476, 477, 478, 479, 480, 481, 482, 483, 484, 485, 486, 487, 488, 489, 490, 491, 492, 493, 494, 495, 496, 497, 498, 499, 500, 501, 502, 503, 504, 505, 553, 554, 555, 556, 557, 558, 559, 560, 561, 562, 563, 564, 565, 566, 567, 568, 569, 570, 571, 572, 573, 574, 575, 576, 577, 578, 579, 580, 581, 582, 583, 584, 585, 586, 587, 588, 589, 590, 591, 592, 593, 594, 595, 596, 597, 598, 599, 600, 601, 602, 603, 604, 605, 606, 607, 608, 609, 610, 611, 612, 613, 614, 615, 616, 617, 618, 619, 620, 621, 622, 623, 624, 625, 626, 627, 628, 629, 682, 683, 684, 685, 686, 687, 688, 689, 690, 691, 692, 693, 694, 695, 696, 697, 698, 750, 751, 752, 753, 754, 755, 756, 757, 758, 804, 805, 806, 807, 808, 809, 810, 811, 812, 813, 814, 815, 816, 817, 818, 819, 820, 821, 822, 823, 824, 825, 826, 827, 828, 829, 830, 831, 832, 833, 834, 835, 836, 837, 838, 839, 840, 841, 842, 843, 844, 845, 846, 847, 848, 849, 850, 851, 852, 853, 854, 855, 856, 857, 858, 859, 860, 861, 862, 863, 864, 865, 866, 867, 868, 869, 870, 871, 872, 873, 874, 875, 876, 877, 878, 879, 880, 881, 882, 883, 884, 888, 889, 890, 891, 892, 893, 894, 895, 896, 897, 898, 899, 900, 901, 902, 903, 904, 905, 906, 907, 908, 909, 910, 911, 912, 913, 914, 915, 949, 950, 951, 952, 953, 954, 955, 956, 957, 958, 959, 960, 961, 962, 963, 964, 965, 966, 967, 968, 969, 970, 971, 972, 973, 974, 975, 1023, 1024, 1025, 1026, 1027, 1028, 1029, 1030, 1031, 1032, 1033, 1034, 1035, 1036, 1037, 1038, 1039, 1040, 1041, 1042, 1043, 1044, 1045, 1046, 1047, 1048, 1049, 1050, 1051, 1052, 1053, 1054, 1055, 1056, 1057, 1058, 1059, 1060, 1061, 1062, 1063, 1064, 1065, 1066, 1067, 1068, 1069, 1070, 1071, 1072, 1073, 1074, 1075, 1076, 1077, 1124, 1125, 1126, 1127, 1128, 1129, 1130, 1131, 1132, 1133, 1134, 1135, 1136, 1137, 1138, 1139, 1140, 1141, 1142, 1143, 1144, 1145, 1146, 1147, 1148, 1149, 1150, 1151, 1152, 1153, 1154, 1155, 1156, 1157, 1158, 1188, 1189, 1190, 1191, 1192, 1193, 1194, 1195, 1196, 1197, 1198, 1199, 1200, 1201, 1202, 1203, 1204, 1205, 1206, 1207, 1208, 1209, 1210, 1211, 1212, 1213, 1214, 1215, 1216, 1217, 1218, 1219, 1220, 1221, 1222, 1223, 1224, 1225, 1226, 1227, 1228, 1229, 1230, 1231, 1232, 1233, 1234, 1235, 1236, 1237, 1238, 1239, 1240, 1241, 1242, 1243, 1244, 1245, 1246, 1247, 1248, 1249, 1250, 1251, 1252, 1253, 1254, 1255, 1256, 1257, 1258, 1259, 1260, 1261, 1262, 1263, 1264, 1265, 1266, 1267, 1268, 1269, 1270, 1271, 1272, 1273, 1274, 1275, 1276, 1277, 1278, 1279, 1280, 1281, 1282, 1283, 1284, 1285, 1286, 1287, 1288, 1289, 1290, 1291, 1292, 1293, 1294, 1295, 1296, 1297, 1298, 1299, 1300, 1301, 1302, 1303, 1304, 1305, 1306, 1307, 1308, 1309, 1310, 1311, 1312, 1313, 1314, 1315, 1320, 1321, 1322, 1323, 1324, 1325, 1326, 1333, 1334, 1335, 1336, 1337, 1338, 1339, 1340, 1341, 1342, 1343, 1344, 1345, 1346, 1347, 1348, 1349, 1350, 1351, 1352, 1353, 1354, 1355, 1356, 1357, 1358, 1359, 1360, 1361, 1362, 1363, 1364, 1365, 1366, 1367, 1368, 1369, 1378, 1379, 1380, 1381, 1382, 1383, 1384, 1385, 1386, 1387, 1388, 1389, 1390, 1391, 1392, 1393, 1394, 1395, 1396, 1397, 1404, 1405, 1406, 1407, 1408, 1409, 1410, 1411, 1412, 1413, 1414, 1415, 1416, 1417, 1418, 1419, 1420, 1421, 1422, 1423]
    #n_4_train = n_4_train[0:800]
    #n_4_test = [1424, 1425, 1426, 1427, 1428, 1429, 1430, 1431, 1432, 1433, 1434, 1435, 1436, 1437, 1438, 1439, 1440, 1441, 1442, 1443, 1444, 1445, 1446, 1447, 1448, 1449, 1450, 1451, 1455, 1456, 1457, 1458, 1459, 1460, 1461, 1462, 1463, 1464, 1465, 1466, 1467, 1468, 1469, 1470, 1471, 1472, 1473, 1474, 1475, 1476, 1477, 1478, 1479, 1480, 1481, 1482, 1483, 1484, 1485, 1486, 1487, 1488, 1489, 1490, 1491, 1492, 1493, 1494, 1495, 1496, 1497, 1498, 1522, 1523, 1524, 1525, 1526, 1527, 1528, 1529, 1530, 1531, 1532, 1533, 1534, 1535, 1536, 1537, 1538, 1539, 1540, 1541, 1548, 1549, 1550, 1551, 1552, 1553, 1554, 1555, 1556, 1557, 1558, 1559, 1560, 1561, 1562, 1563, 1564, 1565, 1566, 1567, 1568, 1569, 1570, 1571, 1572, 1573, 1574, 1575, 1576, 1577, 1578, 1579, 1580, 1581, 1582, 1583, 1584, 1585, 1586, 1587, 1588, 1590, 1591, 1592, 1593, 1594, 1595, 1596, 1597, 1598, 1599, 1600, 1601, 1602, 1603, 1604, 1605, 1606, 1607, 1608, 1609, 1610, 1611, 1612, 1613, 1614, 1615, 1616, 1617, 1618, 1619, 1620, 1621, 1622, 1623, 1624, 1625, 1626, 1627, 1628, 1629, 1630, 1631, 1632, 1633, 1634, 1635, 1636, 1637, 1638, 1639, 1640, 1641, 1642, 1643, 1644, 1645, 1646, 1647, 1648, 1649, 1650, 1651, 1652, 1653, 1654, 1655, 1656]
    #n_4_test = n_4_test[0:100]
    #n_4_test = random.sample(n_4_train,10)
    
    #n_4_train = random.sample(range(0,max_n+1), int(max_n*thrs))
    #n_4_test = [x for x in range(max_n) if x not in n_4_train] 
    
    n_4_train = random.sample(range(0,dim+1), int(dim*thrs))
    n_4_test = [x for x in range(dim) if x not in n_4_train]   

    
    train = all_trajectories[all_trajectories["n_traj"].isin(n_4_train)]
    train = train.sort_values('time').reset_index(drop=True)
    test = all_trajectories[all_trajectories["n_traj"].isin(n_4_test)]
    test = test.sort_values('time').reset_index(drop=True)

    train_data = get_data_in_format(train, dim_frame)
    test_data = get_data_in_format(test, dim_frame)


    return train_data, test_data

In [6]:
train, test = get_data_4_model(trajectories, 0.8, 1)

  0%|          | 0/4959859 [00:00<?, ?it/s]

  0%|          | 0/1260065 [00:00<?, ?it/s]

In [7]:
train.to_csv("shifted_2_1s.csv", mode="a", header=False, index=False)
test.to_csv("shifted_2_1s_test.csv", mode="a", header=False, index=False)


In [6]:
train.to_csv("completo_NOshifted_2_04s.csv", mode="a", header=False, index=False)
test.to_csv("completo_NOshifted_2_04s_test.csv", mode="a", header=False, index=False)
